In [54]:
## https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

## https://github.com/aboSamoor/polyglot/issues/80
# conda install -c conda-forge pyicu morfessor icu -y && pip install pycld2 polyglot


# polyglot download TASK:ner2
# polyglot download TASK:embeddings2

In [55]:
data_path='../../data/'
data_raw_path="../../data/raw/for third propose/"
data_man_path="../../data/manipulate/for third propose/"
model_path="../../model/modeling_third_propose/"
w2v_path="../../data/w2v/"

In [56]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import nltk
from string import digits
import ast

import polyglot
from polyglot.text import Text, Word

In [57]:
# https://polyglot.readthedocs.io/en/latest/Embeddings.html
from polyglot.mapping import Embedding
from polyglot.mapping import CaseExpander, DigitExpander
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors

In [58]:
stop_words_id=open(data_path+"stopwords_id.txt")
stop_words_id=list(set([i[:-1] for i in stop_words_id.readlines()]))

stop_words_en=open(data_path+"stopwords_en.txt")
stop_words_en=list(set([i[:-1] for i in stop_words_en.readlines()]))

stop_words=stop_words_id+stop_words_en

In [59]:
stop_word_add=['nambah', 'byk', 'yaaa', 'ksh', 'bs', 'kl', 'sm', 'smua',
       'seadanya', 'ajh', 'atau', 'mnta', 'klw', 'ato', 'kalo', 'abis',
       'bnyk', 'pokoknya', 'jgn', 'smuanya', 'klo', 'gitu', 'yaaaa',
       'gpp', 'disitu', 'aj', 'hehe', 'jg', 'yng', 'ya', 'habis', 'gada',
       'ajaa', 'klu', 'ush', 'terserah', 'yah', 'salah', 'brp', 'yaa',
       'aja', 'trs', 'tambahin', 'nya', 'tp', 'ajah', 'mnt', 'situ',
       'segitu', 'disatuin', 'satuin', 'gapapa', 'yg', 'gabung', 'trus',
       'biar', 'nyaa', 'klau', 'lbh']
stop_words2=stop_words+stop_word_add

# Get The Data and create our word embedding

In [60]:
# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W7MeoBMzY3E
# https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
# https://github.com/RaRe-Technologies/gensim/issues/814

In [61]:
def remove_one_stops(X):
    X2=[]
    for c in X:
        if len(c)>=2 and c not in stop_words2:
            X2.append(c)
    return X2

In [62]:
from gensim.models import Word2Vec
# define training data

In [63]:
# dtext=pd.read_csv(data_raw_path+"data_text_goshop_2.csv")
# dtag=pd.read_csv(data_raw_path+"data_tag_goshop.csv")

dtext2=pd.read_csv(data_man_path+"data_gocommerce_items_sample_1_cln_w_label3.csv")
dtext2["real_tags"]=dtext2.real_tags.apply(lambda x : set(ast.literal_eval(x)))
# dtag=pd.read_csv(data_man_path+"data_tag_goshop2.csv")
# dtag.to_csv(data_man_path+"data_tag_goshop2.csv",index=False)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
dtext2_food=dtext2[dtext2.type=="food"][["item_list_cln2","merchant_name_cln1","type"]]
dtext2_non_food=dtext2[dtext2.type!="food"][["item_list_cln2","merchant_name_cln1","type"]]

dtext2_food["sentence_used"]=dtext2_food.item_list_cln2+" "+dtext2_food.merchant_name_cln1
# dtext2_food["sentence_used"]=dtext2_food.item_list_cln2
dtext2_non_food["sentence_used"]=dtext2_non_food.item_list_cln2

dtext3=pd.concat([dtext2_food,dtext2_non_food])
# dtext3=pd.concat([dtext2_non_food])
# dtext3=dtext2_non_food[dtext2_non_food.type=="shop"]

In [65]:
dtext3.head()

,item_list_cln2,merchant_name_cln1,type,sentence_used
2799998,nasi paket ayam geprek,geprek bensu,food,nasi paket ayam geprek geprek bensu
2799999,bakso mukidi,mie ayam bakso mukidi,food,bakso mukidi mie ayam bakso mukidi
2800000,italian meat ball stuffed crust regular chessy...,phd,food,italian meat ball stuffed crust regular chessy...
2800001,nasi bebek,bebek sinjay,food,nasi bebek bebek sinjay
2800002,salted egg chicken,sec bowl by rius vernandes,food,salted egg chicken sec bowl by rius vernandes


In [66]:
sentence=[]
# for il in dtext2.sample(15,random_state=100).item_list_cln2.tolist()[:10]:
for il in dtext3[~(dtext3.sentence_used.isnull())].sentence_used.tolist():
    il2=il.split(" ")
#     sentence.append(il2)
    il3=remove_one_stops(il2)
    sentence.append(il3)    

In [67]:
len(sentence)

3599510

In [68]:
# sentence

In [69]:
# model_emb = Word2Vec(sentence,size=100,
#         window=8,min_count=60,workers=5)
model_emb = Word2Vec(sentence,size=90,
        window=8,min_count=40,workers=5)

In [70]:
model_emb.train(sentence, total_examples=len(sentence), epochs=10)

(234761421, 279159060)

In [71]:
model_emb["remote"]

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


array([ 0.11427715,  1.6115043 ,  0.78236306, -1.6907134 , -0.24915132,
        0.35412213,  0.79163283,  0.14339279, -1.6021619 ,  0.9053349 ,
       -0.27305898, -1.5851066 ,  1.3857781 , -1.9324019 ,  0.53924507,
        0.36737087,  0.4550265 ,  3.6157942 , -2.0297515 , -3.0608616 ,
       -0.5734429 , -0.39045066,  0.24151619, -1.8992816 , -0.5449692 ,
       -1.7906028 , -0.91664904,  3.4364398 ,  1.3262398 , -1.6691551 ,
       -1.9648712 ,  0.49161977, -1.8834251 , -2.130431  , -0.7933773 ,
       -1.0450284 , -0.45371664,  0.6407101 ,  0.38878283, -2.360629  ,
        1.3636858 ,  2.4583175 ,  0.16506055, -1.0984727 ,  0.78420293,
        1.0610428 , -0.6237064 ,  1.0974616 ,  1.0192817 , -2.2474856 ,
       -0.7491692 , -0.06259245, -1.4241807 ,  1.8909247 , -0.5780439 ,
        1.060845  ,  1.742925  , -0.22930385,  0.9235678 , -0.17818184,
        2.8464372 ,  1.1480416 , -0.5466136 , -0.8700628 , -1.7159238 ,
        2.2271917 ,  0.9053491 , -1.450653  , -3.408193  , -0.46

In [72]:
# [k[0] for k in model_emb.wv.most_similar(positive="babi",topn=20)]
model_emb.wv.most_similar(positive="ayam",topn=20)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ayan', 0.622320294380188),
 ('atam', 0.5937215089797974),
 ('aym', 0.5900243520736694),
 ('porsiayam', 0.5772808194160461),
 ('goreng', 0.5650893449783325),
 ('bungkusayam', 0.5523208379745483),
 ('nasi', 0.5451222658157349),
 ('komplit', 0.5297495126724243),
 ('porsi', 0.52732914686203),
 ('paha', 0.492574006319046),
 ('usus', 0.49121272563934326),
 ('ayamgoreng', 0.48943841457366943),
 ('rebus', 0.4858480989933014),
 ('potongayam', 0.4856860637664795),
 ('sapi', 0.46591830253601074),
 ('kuah', 0.46457886695861816),
 ('babi', 0.46023789048194885),
 ('terong', 0.4584827721118927),
 ('nasiputih', 0.4578687250614166),
 ('bebek', 0.44995731115341187)]

In [73]:
model_emb.wv.similarity(w1="ayam",w2="chicken")

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.3148526

In [74]:
model_emb.save(w2v_path+'our_vocab3_w_merchant.bin')

In [75]:
1+1

2

In [76]:
c_chick=0
for ss in sentence:
    ww=" ".join(ss)
    if "chicken" in ww:
        c_chick+=1

In [77]:
c_chick

123928